# Quantum SAT solver

In [ ]:
import numpy as np
import math as m
import time
import sys
from itertools import combinations as comb

In [ ]:
from functools import partial
from pycpd import rigid_registration

### Data Treatment

The information we initial have is the 3D coordinates of the points of our graphs.

In [ ]:
class Point( object ):
    def __init__( self, x, y, z, data ):
        self.x, self.y, self.z = x, y, z
        self.data = data

    def __str__ (self):
        return "Ponto do tipo %s. Coordenadas %s %s %s" % (self.data, self.x, self.y, self.z)

In [ ]:
import csv

csv.register_dialect('myDialect',
delimiter = ',',
skipinitialspace=True)

In [ ]:
with open('input_BIAL.csv', 'r') as csvDataFile:
    csvReader = csv.reader(csvDataFile, dialect = 'myDialect')
    for row in csvReader:
        print(row)

The rows 1, 3, 4  and 5 have to change to int or to float:

In [ ]:
database = []

with open('input_BIAL.csv', 'r') as csvDataFile:
    csvReader = csv.reader(csvDataFile, dialect = 'myDialect')

    mol = -1
    molecule_type = ""
    idconf = -1
    
    for idx, row in enumerate(csvReader):
        if (row[0] != 'pdb'):
            # if the row does not have the title
            # then we have one of the molecules
            
            if (molecule_type != row[0]):
                # if the molecule in row[0] is different from 
                # the molecule stored in the variable "molecule_type"
                # identify the new molecule
                print ("Molecule", row[0])
                # update molecule 
                molecule_type = row[0]
                # create space in the list for the molecule
                database.append([])
                mol = mol+1
                idconf= -1
            # (else) we are in the same molecule
            
            if( idconf != int(row[1])):
                # if the conformation number is different 
                # from the one in variable idconf + 1
                # then identify this new conformation
                print("new conformation: ",row[1])
                # update the number of the conformation
                idconf = idconf+1
                # create space in the list for the next conformation 
                database[mol].append([])
            
            #simplify name of the types
            types=""
            if "H" in row[2]:
                types="H"
            if "P" in row[2]:
                types ="P"
            
            print("Adding the point",types,"with coordinates ",row[3],row[4],row[5])
            # add point to the corresponding molecule and conformation in the list
            database[mol][idconf].append(Point(float(row[3]), 
                                               float(row[4]), 
                                               float(row[5]), 
                                               types))

now we have a database of every point in every conformation

database == list of molecules = \[molecule, ..., molecule\]

molecule == list of conformations = \[conformation, ..., conformation\]

conformation == list of points \[point, ..., point\]

database = \[ \[ \[ Points \] \], ..., \[ \[ Points \] \] \]

In [ ]:
for idxMol, mol in enumerate(database):
    print("new molecule")
    for idxConf, conf in enumerate(mol):
        print("new conformation")
        for idxPoint, point in enumerate(conf):
            print (point.x,point.y,point.z,point.data)

### Auxiliary functions for restriction verification

The first definition is the distance (d) between every two points.

$$
d = ((x_2 - x_1)^2 + (y_2 - y_1)^2 + (z_2 - z_1)^2)^{1/2} 
$$

In [ ]:
def dist ( PointA, PointB ):
    xa = PointA.x
    xb = PointB.x
    ya = PointA.y
    yb = PointB.y
    za = PointA.z
    zb = PointB.z 

    final = m.sqrt( (xa-xb)**2 + (ya-yb)**2 + (za-zb)**2 )

    return final

In [ ]:
def sameType (*points): #return true or false whenever the 2 or more points are of the same type or not
    result = True
    type_data = ""
    
    if(len(points)<2):
        raise Exception('Number of points to be compared should be at least 2. The number of points was: {}'.format(len(points)))
    
    for i in points:
        if (type_data == ""):
            type_data = i.data
        
        result = result and (type_data==i.data)
        
    return result

In [ ]:
def possible (point1,point2): 
    
    value = 0

    if (sameType(point1,point2) and (dist(point1,point2)<=2)):
        value = 1
    
    return value

## Pre-Alignment
For only two molecules at a time.

In [ ]:
def convertTolist(conf):
    
    c = []
    t = []
    
    for idx, p in enumerate(conf):
        c.append([])
        c[idx].append(p.x)
        c[idx].append(p.y)
        c[idx].append(p.z)
        t.append(p.data)
    
    return [np.array(c),t]

def unconvert(c,t):
    conf = []
    
    for idx, p in enumerate(c):
        conf.append(Point(p[0],p[1],p[2],t[idx]))
    
    return conf

In [ ]:
def transformation(*confs):
    # X and Y are lists with coordinates of 2 different
    #conformations that we want to "align"
    if (len(confs)==2):
        [c1,t1], [c2,t2] = convertTolist(confs[0]), convertTolist(confs[1])
        
        reg1 = rigid_registration(**{ 'X': c1, 'Y': c2 })
        TY1, _, err1 = reg1.register()
    
        reg2 = rigid_registration(**{ 'X': c2, 'Y': c1 })
        TY2, _, err2 = reg2.register()
        
        if(err1 < err2):
            result = unconvert(c1,t1), unconvert(TY1,t2)
        else:
            result = unconvert(TY2,t1), unconvert(c2,t2)
    
    return result

## Creation of the structures required for the SAT solver
#### Vector D and P
For only two molecules at a time.

In [ ]:
#Only considering edges between points of differents conformations and molecules!
def combinations (*molecules):
    
    numberPoints = []
    for mol in molecules:
        numberPoints.append(len(mol[0]))
    
    vectorD = np.zeros(np.prod(numberPoints))
    vectorP = np.zeros(np.prod(numberPoints))
    
    listvectorsD = []
    listvectorsP = []
    
    size = len(molecules)

    if(size==2):
        for idxConf1, conf1 in enumerate(molecules[0]):
            for idxConf2, conf2 in enumerate(molecules[1]):
                nconf1, nconf2 = transformation(conf1,conf2)
                for idxP1, p1 in enumerate(nconf1):
                    for idxP2, p2 in enumerate(nconf2):
                        a = dist(p1,p2)
                        b = possible(p1,p2)
                        vectorD[idxP2+(idxP1 * len(nconf2))] = a
                        vectorP[idxP2+(idxP1 * len(nconf2))] = b 
                        
                listvectorsD.append(vectorD)
                listvectorsP.append(vectorP)
                vectorD = np.zeros(np.prod(numberPoints))
                vectorP = np.zeros(np.prod(numberPoints))
    return listvectorsD, listvectorsP

In [ ]:
listDs = []
listPs = []
listDs, listPs = combinations(database[1],database[2])

Multiply the distances by 10.

In [ ]:
D = [x*10 for x in listDs[0]]
P = [ x for x in listPs[0]]

Create a solution to be tested in the Ising Hamiltonian for a sanity check.

In [ ]:
I = [ x for x in np.ones(len(D))]
print(D,P,I)

## Quantum SAT

In [ ]:
import operator
import matplotlib.pyplot as plt

import sys
if sys.version_info < (3, 6):
    raise Exception('Please use Python version 3.6 or greater.')

# Qiskit packages
from qiskit import BasicAer
from qiskit import IBMQ
from qiskit.quantum_info import Pauli
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import NumPyEigensolver, VQE
from qiskit.aqua.components.variational_forms import RY
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.operators import WeightedPauliOperator

import logging
from qiskit.aqua._logging import set_logging_config, build_logging_config

import qiskit.tools.jupyter
%qiskit_version_table

### Evaluate the quantum solution for each pair of conformations, that is, each pair of vector D and P

Each position of the vector list's correspond to a diferent pair of conformations.


### Construct the Ising Hamiltonian

#### for 2 molecules

$$
H = \sum_{i \sim j}^{N} d_{ij} x_{ij} - 
\sum_{i \sim j}^{N} x_{ij} + 
A \sum_{i \sim j}^{N} \Big(p_{ij} - x_{ij} \Big)^2 + 
B \sum_{i}^N \Big( M-1 - \sum_{j\sim i}^{N} x_{ij} \Big)^2
$$


where:

$\it N$ is the number of points in a conformation. Every conformation of a given molecule has de same number of points, so, N can also be denoted as the number of points in a molecule

$\it M$ is the number of different molecules to align 

$\it A$ is a big enough parameter.

$\it d_{ij}$ is a vector representing the distance of each edge

$\it p_{ij}$ is a binary vector representing the possibility of each edge

$\it x_{ij}$ is our final vector, to optimize



### From Hamiltonian to QP formulation in the z basis, only considering the objective function
#### for 2 molecules

In the vector ${\bf z}$, where $x_i \rightarrow \frac{(1-z_i)}{2}$, $H$ can be written as follows.

$$
\min_{{\bf z}\in \{0,1\}^{N_{mol1}xN_{mol2}}} \mathbf{H} = \frac{1}{2}\sum_{i \sim j}^{N} d_{ij} -\frac{1}{2}\sum_{i \sim j}^{N} d_{ij}z_{ij} -\frac{1}{2}\sum_{i \sim j}^{N} 1 + \frac{1}{2}\sum_{i \sim j}^{N} z_{ij} 
$$

That is:
$$
\mathbf{H} =  \Big( \frac{1}{2}( \mathbf{1} - \mathbf{d} ) \Big)^T \mathbf{z} + \frac{1}{2}(\mathbf{d}-\mathbf{1}) \\
= \mathbf{g}^T \mathbf{z} + \mathbf{c}
$$

The QP formulation of the Ising Hamiltonian is ready for the use of VQE. 

#### Login into IBM platafomrm

In [ ]:
token = '' #insert token inside the quotes
IBMQ.enable_account(token)

#### Complete acoording to your HUB credentials or mantain as it is

In [ ]:
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')

See the available backends.

In [ ]:
provider.backends()

### Quantum solution from the ground up

By use of Qiskit, derive the solution from the ground up, using a class QuantumOptimizer that encodes the quantum approach to solve the problem. Then, instantiate it and solve it.

In [ ]:
class QuantumOptimizer:
    def __init__(self, d, p, max_trials=1000):

        self.d = d
        self.p = p
        self.size = len(d)
        self.max_trials = max_trials

    def binary_representation(self,x_sol=[]):

        p = self.p
        d = self.d
        size = self.size
        ones = np.ones(size)

        # g defines the contribution from the individual variables
        g = [x / 2 for x in np.subtract(ones,d)]
        # c is the constant offset
        c = np.sum([x / 2 for x in np.subtract(d,ones)])

        try:
            fun = lambda x: np.dot(d,x) - np.dot(ones,x)
            cost = fun(x_sol)
        except:
            cost = 0
        
        return g, c, cost
    
    def construct_hamiltonian(self):

        p = self.p
        d = self.d
        size = self.size

        N = size  # number of qubits
        
        gz,cz, _ = self.binary_representation() # already in the Z-basis
        
        # Getting the Hamiltonian in the form of a list of Pauli terms

        pauli_list = []
        for i in range(N):
            if gz[i] != 0:
                wp = np.zeros(N)
                vp = np.zeros(N)
                vp[i] = 1
                pauli_list.append((gz[i], Pauli(vp, wp)))

        pauli_list.append((cz, Pauli(np.zeros(N), np.zeros(N))))

        return cz, pauli_list

    def check_hamiltonian(self):

        cz, op = self.construct_hamiltonian()
        Op = WeightedPauliOperator(paulis=op)

        qubitOp, offset = Op, 0
        
        # Making the Hamiltonian in its full form and getting the lowest eigenvalue and eigenvector
        
        result0 = NumPyEigensolver(operator=qubitOp).run()
        result = result0['eigenstates'].to_matrix(massive=True)
        
        quantum_solution = self._q_solution(np.real(result[0]).tolist(),self.size)
        
        ground_level = np.real(result0['eigenvalues'][0]) + offset

        return quantum_solution, ground_level

    def vqe_solution(self):

        cz, op = self.construct_hamiltonian()
        Op = WeightedPauliOperator(paulis=op)

        qubitOp, offset = Op, cz

        aqua_globals.random_seed = 10598
        
        num_qubits = qubitOp.num_qubits
        var_form = RY(qubitOp.num_qubits, depth=5, entanglement='linear')
        optimizer = SPSA(max_trials=self.max_trials)
        algo = VQE(qubitOp, var_form, optimizer)
        
        #choose acoording to the available backend by your credentials
        backend = provider.get_backend('ibmq_qasm_simulator')
        #backend = provider.get_backend('ibmq_cambridge')
        
        quantum_instance = QuantumInstance(backend,
                                           seed_simulator=aqua_globals.random_seed,
                                           seed_transpiler=aqua_globals.random_seed,
                                           skip_qobj_validation=False)
        
        print("           Running")
        start = time.time()
        result = algo.run(quantum_instance)
        print("           time taken (seconds):",time.time() - start)
        
        quantum_solution_dict = result['eigenstate']

        q_s = max(quantum_solution_dict.items(), key=operator.itemgetter(1))[0]
        quantum_solution= [int(chars) for chars in q_s]
        quantum_solution = np.flip(quantum_solution, axis=0)

        _,_,level = self.binary_representation(x_sol=quantum_solution)
        return quantum_solution_dict, quantum_solution, level

    def _q_solution(self, v, N):
        
        for x in range(len(v)):
            if v[x] == max(v):
                index_value = x
                break
                
        string_value = "{0:b}".format(index_value)

        while len(string_value)<N:
            string_value = '0'+string_value

        sol = list()
        for elements in string_value:
            if elements == '0':
                sol.append(0)
            else:
                sol.append(1)

        sol = np.flip(sol, axis=0)

        return sol

### Step 1

Instantiate the quantum optimizer class with the parameters. It was used 100 as the max_trials.

In [ ]:
# Instantiate the quantum optimizer class with parameters: 
quantum_optimizer = QuantumOptimizer(D,P,100)

### Step 2

Encode the problem as a binary formulation (IH-QP).

Sanity check: make sure that the binary formulation in the quantum optimizer is correct (i.e., yields the same cost given the same solution).

In [ ]:
# Check if the binary representation is correct
g,c,binary_cost = quantum_optimizer.binary_representation(I)
print(binary_cost)

### Step 3

Encode the problem as an Ising Hamiltonian in the Z basis. 

Sanity check: make sure that the formulation is correct (i.e., yields the same cost given the same solution)

#### DO NOT RUN in the current Qiskit version

In [ ]:
ground_state, ground_level = quantum_optimizer.check_hamiltonian()
print(ground_state,ground_level)

### Step 4

Solve the problem via VQE. N.B. Depending on the number of qubits, the state-vector simulation can can take a while; for example with 12 qubits, it takes more than 12 hours. Logging useful to see what the program is doing.


In [ ]:
quantum_dictionary, quantum_solution, quantum_cost = quantum_optimizer.vqe_solution()

print(quantum_solution, quantum_cost)